In [1]:
import pandas as pd
data2 = pd.read_csv("data/data2.csv")
data3 = pd.read_csv("data/data3.csv")
data4 = pd.read_csv("data/data4.csv")
data5 = pd.read_csv("data/data5.csv")

In [2]:
data2 = data2.drop('Unnamed: 0', 1)
data3 = data3.drop('Unnamed: 0', 1)
data4 = data4.drop('Unnamed: 0', 1)
data5 = data5.drop('Unnamed: 0', 1)

In [3]:
full_data = pd.concat([data2,data3, data4, data5])
full_data = full_data.drop_duplicates()
full_data = full_data.reset_index(drop=True)

In [4]:
full_data.head(5)

,tags,coordinates,time,fav_count,review,country_code,lon,lat
0,"['vegan', 'veganfood', 'veganfoodshare', 'vega...","[-76.1105804, 36.8410797]",Thu Nov 08 14:11:52 +0000 2018,0.0,You need to gobble these donuts up right now! ...,US,-76.11058,36.84108
1,"['Repost', 'vegansofig', 'veganlifestyle', 'wh...","[-76.8882299, 40.26374]",Thu Nov 08 14:21:46 +0000 2018,0.0,Delish vegan protein bowl #Repost @overexposed...,US,-76.88823,40.26374
2,[],"[-72.89356, 41.67243]",Thu Nov 08 14:22:27 +0000 2018,0.0,Pop’d it! 🤩🍫🍿\n•\nStop in Today from 11-6pm🍭\n...,US,-72.89356,41.67243
3,"['vegansofig', 'veganlifestyle', 'whatvegansdr...","[-76.8882299, 40.26374]",Thu Nov 08 14:24:08 +0000 2018,0.0,Vegan Vanilla Donuts with Oreos smothered in C...,US,-76.88823,40.26374
4,[],"[-82.9996, 39.9624]",Thu Nov 08 14:25:40 +0000 2018,0.0,"I have a LONG WAY TO Go, but hit 315 for 3 set...",US,-82.99960,39.96240


In [5]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [7]:
import numpy as np
# We create a column with the result of the analysis:
full_data['SA'] = np.array([ analize_sentiment(tweet) for tweet in full_data['review'] ])

# We display the updated dataframe with the new column:
display(full_data.head(5))

,tags,coordinates,time,fav_count,review,country_code,lon,lat,SA
0,"['vegan', 'veganfood', 'veganfoodshare', 'vega...","[-76.1105804, 36.8410797]",Thu Nov 08 14:11:52 +0000 2018,0.0,You need to gobble these donuts up right now! ...,US,-76.11058,36.84108,1
1,"['Repost', 'vegansofig', 'veganlifestyle', 'wh...","[-76.8882299, 40.26374]",Thu Nov 08 14:21:46 +0000 2018,0.0,Delish vegan protein bowl #Repost @overexposed...,US,-76.88823,40.26374,0
2,[],"[-72.89356, 41.67243]",Thu Nov 08 14:22:27 +0000 2018,0.0,Pop’d it! 🤩🍫🍿\n•\nStop in Today from 11-6pm🍭\n...,US,-72.89356,41.67243,1
3,"['vegansofig', 'veganlifestyle', 'whatvegansdr...","[-76.8882299, 40.26374]",Thu Nov 08 14:24:08 +0000 2018,0.0,Vegan Vanilla Donuts with Oreos smothered in C...,US,-76.88823,40.26374,0
4,[],"[-82.9996, 39.9624]",Thu Nov 08 14:25:40 +0000 2018,0.0,"I have a LONG WAY TO Go, but hit 315 for 3 set...",US,-82.99960,39.96240,-1


In [8]:
# We construct lists with classified tweets:

pos_tweets = [ tweet for index, tweet in enumerate(full_data['review']) if full_data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(full_data['review']) if full_data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(full_data['review']) if full_data['SA'][index] < 0]

In [9]:
# We print percentages:

print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(full_data['review'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(full_data['review'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(full_data['review'])))

Percentage of positive tweets: 54.33687744823727%
Percentage of neutral tweets: 34.61108002238388%
Percentage de negative tweets: 11.052042529378847%


In [10]:
import gmaps
import gmaps.datasets

gmaps.configure()
#gmaps.configure()
positive_df = full_data[full_data['SA'] == 1]
positive_df = positive_df[['lat', 'lon']]

negative_df = full_data[full_data['SA'] == -1]
negative_df = negative_df[['lat', 'lon']]

neutral_df = full_data[full_data['SA'] == 0]
neutral_df = neutral_df[['lat', 'lon']]

positive_layer = gmaps.symbol_layer(positive_df, fill_color='rgba(0, 150, 0, 0.4)',stroke_color='rgba(0, 150, 0, 0.4)', scale=2)
negative_layer = gmaps.symbol_layer(negative_df, fill_color='rgba(200, 0, 0, 0.4)',stroke_color='rgba(200, 0, 0, 0.4)', scale=2)
neutral_layer = gmaps.symbol_layer(neutral_df, fill_color='rgb(255,255,153)',stroke_color='rgb(255,255,153)', scale=2)
fig = gmaps.figure()
fig.add_layer(negative_layer)
fig.add_layer(neutral_layer)
fig.add_layer(positive_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Import the necessary libraries
import pandas as pd
import gmplot
# For improved table display in the notebook
from IPython.display import display

latitudes = full_data["lat"]
longitudes = full_data["lon"]

#gmap = gmplot.GoogleMapPlotter(50.0522, -90.2437, 4)
gmap = gmplot.GoogleMapPlotter(50.0522, -90.2437, 4)

# Overlay our datapoints onto the map
gmap.heatmap(latitudes, longitudes)
gmap.draw("heatmap_tweets.html")